# 0. Background

### Goal
Forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer. <br>

#### Instructions
Specifically, build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.<br>

**Submission**
For each id in the test set, you must predict a value for the sales variable. The file should contain a header and have the following format:<br>
id,sales<br>
3000888,0.0<br>
3000889,0.0<br>
3000890,0.0<br>
3000891,0.0<br>
3000892,0.0<br>
etc.


### Provided...
-> Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices. <br>
-> Wages in the public sector are paid every two weeks on the 15th and on the last day of the month. Supermarket sales could be affected by this. <br>
-> A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake. <br>
-> Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. <br>
-> Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge. <br>
-> Additional holidays are days added to a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday). <br>

**-> Evaluation metric: RMSLE**<br>
The RMSLE is calculated as:<br>
$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(1+y^i)−log(1+y_{i}))^2}$<br>
where:<br>

$n$
 is the total number of instances,<br>
$y^i$
 is the predicted value of the target for instance (i),<br>
$yi$
 is the actual value of the target for instance (i), and,<br>
$log$
 is the natural logarithm.<br>

### Additional...
-> Train dataset period: 2013-01-01 to 2017-08-15 <br>
-> Test dataset period: 2017-08-16 to 2017-08-31 <br>
-> Christmas day is missing from the train dataset for a few years. The stores are likely closed on Christmas. <br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Load data

In [ ]:
# Load data
df_train = pd.read_csv('train.csv', parse_dates=["date"])
df_trans = pd.read_csv('transactions.csv', parse_dates=["date"])
df_stores = pd.read_csv('stores.csv')
df_oil = pd.read_csv('oil.csv', parse_dates=["date"]).rename(columns={"dcoilwtico": "oil"})
df_holi = pd.read_csv('holidays_events.csv', parse_dates=["date"])
df_sub = pd.read_csv('sample_submission.csv')
df_test = pd.read_csv('test.csv', parse_dates=["date"])

# 2. Explore data 

## df_train

### Key takeaways
1. Top 5 product families: Grocery I, Beverages, Produce, Cleaning, Dairy.
2. Promotions and sales have a positive linear relationships.
3. Promotions are most common on Wednesdays, Fridays, and holidays
4. Promotions (total sum) started mid 2015 and have steadily increased since then.

In [ ]:
# Store count = 54
df_train.store_nbr.nunique()

# Same stores in training and test sets
df_train.store_nbr.unique() == df_test.store_nbr.unique()

# Family of products count = 33
df_train.family.nunique()

# Same family of products in training and test sets
df_train.family.unique() == df_test.family.unique()

In [ ]:
# Total sales by store
dfStoreSales = df_train.groupby(["store_nbr"]).sales.sum().reset_index().sort_values(by = 'sales', ascending=False)
dfStoreSales.describe()

In [ ]:
# Is every product family reported for every date and store_nbr?
# Store numbers - Every one (54 total) reported for every date in training set
df_store_nbr = df_train.groupby(['date',])['store_nbr'].nunique().reset_index()
df_store_nbr[df_store_nbr['store_nbr'] != 54]

# Product families - Every one (33 total) reported for every date and store number
df_family = df_train.groupby(['date', 'store_nbr'])['family'].nunique().reset_index()
df_family[df_family['family'] != 33]

# How about in the test set?
# Store numbers - Every one (54 total) reported for every date in training set
df_store_nbr2 = df_test.groupby(['date',])['store_nbr'].nunique().reset_index()
df_store_nbr2[df_store_nbr2['store_nbr'] != 54]

# Product families - Every one (33 total) reported for every date and store number
df_family2 = df_test.groupby(['date', 'store_nbr'])['family'].nunique().reset_index()
df_family2[df_family2['family'] != 33]

In [ ]:
# Most popular
# Most popular product families by sales - Top 5: Grocery I, Beverages, Produce, Cleaning, Dairy
df_train.groupby(['family'])['sales'].sum().sort_values(ascending=False)

# Most popular product families by sales by store - Top 5: Beverages, Cleaning, Grocery I, Produce, Dairy
# Little variation by store. Only 13 top 5 entries not in list above.
df_salesFamilyStore = df_train.groupby(['store_nbr', 'family'])['sales'].sum().reset_index()

PopFamily = []
for i in df_salesFamilyStore.store_nbr.unique():
    PopFamily.append(df_salesFamilyStore[df_salesFamilyStore['store_nbr'] == i].nlargest(5, 'sales'))
dfPopFamily = pd.concat(PopFamily)

dfPopFamily.groupby(['family'])['family'].count().sort_values(ascending=False)

In [ ]:
# Most popular promotion items - Top 5: Grocery I, Produce, Beverages, Dairy, Cleaning
df_train.groupby(['family'])['onpromotion'].sum().sort_values(ascending=False)

# Most popular product families by promotion by store - Top 6 (only 6): Beverages, Grocery I, Produce, Cleaning, Dairy, Deli
df_promoFamilyStore = df_train.groupby(['store_nbr', 'family'])['onpromotion'].sum().reset_index()

promoPopFamily = []
for i in df_promoFamilyStore.store_nbr.unique():
    promoPopFamily.append(df_promoFamilyStore[df_promoFamilyStore['store_nbr'] == i].nlargest(5, 'onpromotion'))
dfPromoPopFamily = pd.concat(promoPopFamily)

dfPromoPopFamily.groupby(['family'])['family'].count().sort_values(ascending=False)

In [ ]:
# Is there a relationship between promotions and sales?
# Generally, there is a positive linear relationship between promotions and sales.
# However, product families with few promotions and sales deviate from this relationship.
dfPlotPromoSales = (df_train.groupby(['family'])[['sales', 'onpromotion']]
                    .sum()
                    .sort_values(by='onpromotion'))

x = dfPlotPromoSales['onpromotion']
y = dfPlotPromoSales['sales']

fig = plt.figure(figsize=(5,5))
plt.scatter(x, y)

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.xlabel('sales - sum')
plt.ylabel('onpromotion - sum')

subax = fig.add_axes([0.65, 0.2, 0.2, 0.2])
x_labelsize = subax.get_xticklabels()[0].get_size()
y_labelsize = subax.get_yticklabels()[0].get_size()
subax.xaxis.set_tick_params(labelsize=6)
subax.yaxis.set_tick_params(labelsize=6)
subax.scatter(x[:15], y[:15])

plt.show()

print("Spearman Correlation between Sales and Onpromotion by product family: {:,.4f}".format(dfPlotPromoSales.corr("spearman").sales.loc["onpromotion"]))

In [ ]:
# Are there certain dates popular for promotions?
# Promotions have increased in popularity over time, especially since mid 2015.
dfPlotPromoDates = df_train.groupby(['date'])['onpromotion'].sum().sort_values(ascending=False).reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDates['date'], dfPlotPromoDates['onpromotion'])

In [ ]:
# What if I disregard year?
# Looks like promotions spike in the middle and at the end of the year.
# But this might not be true every year since there are many more promotions
# in the later years of the training set.
df_train2 = df_train.copy()

df_train2['month-day'] = df_train2['date'].apply(lambda x: x.replace(year = 2020))

dfPlotPromoMonthDay = df_train2.groupby(['month-day'])['onpromotion'].sum().sort_values(ascending=False).reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoMonthDay['month-day'], dfPlotPromoMonthDay['onpromotion'])

In [ ]:
# Look at promotions on a yearly basis
# 2013 - no promotions
# 2014-2018 - looks like intraweek variability
# Not too much trend in time of year.
dfPlotPromoYear = dfPlotPromoDates[(dfPlotPromoDates['date'] >= '2017-01-01')
                                   & (dfPlotPromoDates['date'] < '2018-01-01')]
fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoYear['date'], dfPlotPromoYear['onpromotion'])
plt.show()

In [ ]:
# Look at promotions by day of week
# Wednesdays and Fridays have nearly double the promotions of other days
df_train2['day'] = df_train2['date'].apply(lambda x: x.isoweekday())
dfPlotPromoDay = df_train2.groupby(['day'])['onpromotion'].sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDay['day'], dfPlotPromoDay['onpromotion'])
plt.show()

In [ ]:
# How about promotions on public sector paydays (15th and last day of month)
# No trend here. Promotions drop on the 31st since some months don't have 31 days...
# ...I'm looking at the sum of promotions over all dates in the training set
df_train2['day-date'] = df_train2['date'].dt.day

dfPlotPromoDayDate = df_train2.groupby(['day-date'])['onpromotion'].sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDayDate['day-date'], dfPlotPromoDayDate['onpromotion'])
plt.show()

In [ ]:
# How about promotions on national holidays
# ~40% more promotions on national and all holidays than other days

# Just national holidays
df_train3 = pd.merge(df_train2, df_holi[df_holi['locale'] == 'National'][['date', 'type']], how='left', on='date')
# All holidays
# df_train3 = pd.merge(df_train2, df_holi[['date', 'type']], how='left', on='date')

df_train3['holiday'] = 1
df_train3.loc[df_train3['type'].isna(), 'holiday'] = 0

dfPlotPromoHoliday = df_train3.groupby(['holiday']).agg({'onpromotion': 'sum', 'date':'nunique'}).reset_index()

dfPlotPromoHoliday['norm'] = dfPlotPromoHoliday['onpromotion'] / dfPlotPromoHoliday['date']

dfPlotPromoHoliday.loc[1, 'norm'] / dfPlotPromoHoliday.loc[0, 'norm']

## df_trans

### Key takeaways
1. Eight of the 54 stores seem to have opened after the start date of the training set (2013-01-01).
2. The capital city (Quito) and state (Pinchincha) are home to 33% and 35% of total stores, respectively.
3. 75% (12 of 16) of states only have stores in one city.
4. 43% (7 of 16) of states only have one store.
5. Transactions spike at the end of the year.
6. Transactions decline from the start of the month to about the 10th, then rise.
7. Transactions spike on Saturdays and Sundays.

In [ ]:
# First date with transactions by store
# Only 8 of 54 stores haven't logged transactions since the beginning of the training set
# Those 8 stores must have been opened after 2013-01-01
dfFirstTrans = df_trans.groupby('store_nbr')['date'].min().reset_index()

dfFirstTrans.groupby('date')['store_nbr'].count()

In [ ]:
# Stores with most transactions
dfTransMax = df_trans.groupby('store_nbr')['transactions'].sum().sort_values(ascending=False).reset_index()

# Store with most transactions has 26x more than store with least transactions
dfTransMax.loc[0, 'transactions'] / dfTransMax.loc[53, 'transactions']

dfTransMax.describe()

In [ ]:
# Transactions by locale
dfTransGeo = pd.merge(dfTransMax, df_stores, how='left', on='store_nbr')

# Group by city
# Stores in Cayambe (1) and Quito (18) average the most transactions per store
dfTransCity = dfTransGeo.groupby(['city']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransCity['norm'] = dfTransCity['transactions'] / dfTransCity['store_nbr']
dfTransCity.sort_values(by='norm', ascending=False)

# Group by state
# Stores in the capital state (Pichincha) average ~25% more transactions than the second most state
dfTransState = dfTransGeo.groupby(['state']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransState['norm'] = dfTransState['transactions'] / dfTransState['store_nbr']
dfTransState.sort_values(by='norm', ascending=False)

# 1/3 of stores are in Quito
dfTransCity[dfTransCity['city'] == 'Quito']['store_nbr'] / dfTransCity.store_nbr.sum()

# 35% are in the state of the capital city, Quito (Pichincha)
dfTransState[dfTransState['state'] == 'Pichincha']['store_nbr'] / dfTransState.store_nbr.sum()

In [ ]:
# Group by type
# There are only 5 store types. Type A averages more than double the number of transactions
# per store than second place Type D
dfTransType = dfTransGeo.groupby(['type']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransType['norm'] = dfTransType['transactions'] / dfTransType['store_nbr']
dfTransType.sort_values(by='norm', ascending=False)

# Group by cluster
# There are 17 clusters with 1-7 stores each. The average number of transactions
# per store per cluster varies wildly (2.8 +/- 1.6).
dfTransClus = dfTransGeo.groupby(['cluster']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransClus['norm'] = dfTransClus['transactions'] / dfTransClus['store_nbr']
dfTransClus.sort_values(by='norm', ascending=False)

dfTransClus.describe()

# States and cities can have stores of different types
dfTransGeo.groupby(['state', 'city', 'type']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()

In [ ]:
# 75% of states only have stores in one city
dfCityState = dfTransGeo.groupby(['state',]).agg({'store_nbr': 'nunique', 'city': 'nunique'}).reset_index()
dfCityState[dfCityState['city'] == 1]['state'].count() / dfCityState.state.count()

# ~43% of states only have one store
dfCityState[dfCityState['store_nbr'] == 1]['state'].count() / dfCityState.state.count()

In [ ]:
# Transactions over time
# Steady increase with spikes at end of year
df_transDate = df_trans.groupby('date').transactions.sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(df_transDate['date'], df_transDate['transactions'])
plt.show()

In [ ]:
# Transactions by month
df_transDate['year-month'] = df_transDate['date'].apply(lambda x: x.replace(day = 1))

# Steady increase with spikes at end of year
df_transYearMonth = df_transDate.groupby('year-month').transactions.sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(df_transYearMonth['year-month'], df_transYearMonth['transactions'])
plt.show()

In [ ]:
# Average transactions by day of month
# Transactions decline from the start of the month to about the 10th, then rise.
df_transDate = df_trans.groupby('date').transactions.sum().reset_index()
df_transDate['day'] = df_transDate.date.dt.day
df_transDay = df_transDate.groupby('day').transactions.mean().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(df_transDay['day'], df_transDay['transactions'])
plt.show()

In [ ]:
# Transactions by day of week
# Transactions spike on Saturdays and Sundays.
df_transDate['dayOfWeek'] = df_transDate.date.apply(lambda x: x.isoweekday())
df_transDayOfWeek = df_transDate.groupby('dayOfWeek').transactions.mean().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(df_transDayOfWeek['dayOfWeek'], df_transDayOfWeek['transactions'])
plt.show()

## df_oil

### Key takeaways
1. Oil prices moved from an average of 99 (dollars? per unit) to 47 (dollars? per unit) suddenly at the end of 2014.
2. Prices were relatively stable otherwise with the exception of 3 or 4 swings.
3. 43 dates are missing a daily oil price.
4. Weekends are not included in dataset.

In [ ]:
# Plot oil prices
x = df_oil['date']
y = df_oil['oil']

fig = plt.figure(figsize=(5,5))
plt.scatter(x, y)

# Average 'high' price
x1 = x[:450]
y1 = [y[:450].mean()]*450
plt.plot(x1, y1,"r--")
plt.text(16350, 98, round(y1[0], 2), color='red')

# Average 'low' price
x2 = x[500:]
y2 = [y[500:].mean()]*718
plt.plot(x2, y2,"r--")
plt.text(16200, 46.5, round(y2[0], 2), color='red')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Daily Oil Price')

In [ ]:
# Check for dates missing a daily price
# 43 dates missing a daily price
# Weekends are not included in dataset
df_oil[df_oil['oil'].isna()].date.count()

# Weekdays 5 and 6 (Saturday and Sunday) are missing from the dataset
df_oil.date.dt.weekday.reset_index().groupby(['date'])['date'].count()

## df_holi

### Key definitions
1.  A holiday on a row with column "transferred" equal to True was celebrated on a different date. <br>
    This date was treated like a regular day. The corresponding row with column "type" equal to Transfer <br>
    contains the date the holiday was celebrated.
2. "type" equal to Bridge is an additional day off added to the holiday.
3. "type" equal to Work Day is a day worked that is not normally worked (e.g., Saturday) <br>
    to make up for  a Bridge day.

### Key takeaways:
1. Most holidays are at the national or local level.
2. Holidays/events by year are not stable due to events. 2012 is light. <br>
    2014 has many events related to the World Cup. 2016 has many events related to the earthquake.
3. Most holidays are in the summer months or at the end of the year.
4. Holiday counts dip on Tuesdays and Wednesdays are are relatively stable on other days of the week.

In [ ]:
# Breakdown of holidays by type
df_holi.groupby(['type'])['date'].count()

# Exclude multiple holidays on same date
df_holi.groupby(['type'])['date'].nunique()

In [ ]:
# Count of transferred holidays: 12
len(df_holi[df_holi['transferred'] == True])

# Were all holidays marked as "transferred" = True celebrated elsewhere?
# Yes
df_holi[df_holi['transferred'] == True]
df_holi[df_holi['type'] == 'Transfer']

len(df_holi[df_holi['transferred'] == True]) == len(df_holi[df_holi['type'] == 'Transfer'])

In [ ]:
# Count of "type" equal to Bridge and Work Day both equal to 5.
# Are all five of each coordinated pairs? 
# Yes
df_holi[df_holi['type'] == 'Bridge']
df_holi[df_holi['type'] == 'Work Day']

In [ ]:
# Breakdown of holidays by locale
# Includes 12 duplicates due to transfers
df_holi.groupby(['locale'])['date'].count()

In [ ]:
# Is the count of holidays/events per year relatively stable?
# No. Extra events in 2014 due to the World Cup and in 2016 due to the earthquake.
# 2012 is a light year.
df_holiYear = df_holi.copy()
df_holiYear['year'] = df_holiYear.date.dt.year
df_holiYear.groupby(['year']).date.count()

In [ ]:
# Holidays/events by month
# Most holidays in summer months or at end of year
df_holiYear['month'] = df_holiYear.date.dt.month
df_holiYear.groupby(['month']).date.count()

In [ ]:
# Holidays/events by day of week
# Count dips slightly on Tuesdays and Wednesdays
df_holiYear['day'] = df_holiYear['date'].apply(lambda x: x.isoweekday())
df_holiYear.groupby(['day']).date.count()

# 3. Prepare data

In [ ]:
# Fill in missing dates in training set
# First training data date = 2013-01-01
train_start = df_train.date.min()

# Last training data date = 2017-08-15
train_end = df_train.date.max()

# Check for missing dates (discontinuities) in training data -> Christmas Day
missing_dates = pd.date_range(train_start, train_end).difference(df_train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()
print('Dates missing from training dataset: ', missing_dates)

# Stores are likely closed on Christmas. Add those dates to datset and fill in
# sales and onpromotion with 0s
# Reindex training data
multi_idx = pd.MultiIndex.from_product(
    [pd.date_range(train_start, train_end), df_train.store_nbr.unique(), df_train.family.unique()],
    names=["date", "store_nbr", "family"],
)
df_train = df_train.set_index(["date", "store_nbr", "family"]).reindex(multi_idx).reset_index()

# Fill missing values with 0s
df_train[["sales", "onpromotion"]] = df_train[["sales", "onpromotion"]].fillna(0.)
df_train.id = df_train.id.interpolate(method="linear") # interpolate linearly as a filler for the 'id'

In [ ]:
# Check for continuity in transactions dataset
expected_trans = ((train_end - train_start).days + 1) * df_train.store_nbr.nunique()
total_trans = len(df_trans)
zero_trans = df_train.groupby(['date', 'store_nbr'])['sales'].sum().eq(0).sum()
miss_trans = expected_trans - total_trans - zero_trans

print('Expected entries: ', expected_trans)
print('Total entries:    ', total_trans)
print('Store-days with no entry: ', zero_trans)
print('Missing store-day entries: ', miss_trans)

In [ ]:
# Coordinate dates, sales, and transactions
# Total sales per date for each store
dfStoreSalesDaily = df_train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# Reindex transaction data
df_trans = df_trans.merge(
    dfStoreSalesDaily,
    on=["date", "store_nbr"],
    how="outer",
).sort_values(["date", "store_nbr"], ignore_index=True)

# Fill missing values with 0s for days with zero sales
df_trans.loc[df_trans.sales.eq(0), "transactions"] = 0.
df_trans = df_trans.drop(columns=["sales"])

# Fill remaining missing values using linear interpolation
df_trans.transactions = df_trans.groupby("store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

In [ ]:
# Fill in missing dates (weekends) from oil data
# Reindex oil data
oil_start = df_oil.date.min()
oil_end = df_oil.date.max()

df_oil = df_oil.merge(
    pd.DataFrame({"date": pd.date_range(oil_start, oil_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

# Fill missing values using the last price since the market is closed.
df_oil.oil = df_oil.oil.ffill()
# Then backfill since first date is 2013-01-01 and the market is closed.
df_oil.oil = df_oil.oil.bfill()

# Fill missing values using linear interpolation
# df_oil.oil = df_oil.oil.interpolate(method="linear", limit_direction="both")

In [ ]:
# Streamline holiday/events data
# Remove all extraneous text from description
df_holi['description'] = df_holi.apply(
    lambda x: x.description.lower().replace(x.locale_name.lower(), ""), axis=1,
).replace(
    r"[+-]\d+|\b(de|del|traslado|recupero|puente|guayaquil|santo domingo|cuenca|-)\b", "", regex=True,
).replace(
    r"\s+|-", " ", regex=True,
).replace(
    r'.* futbol .*', "futbol", regex=True,
).replace(
    r'terremoto .*', "terremoto", regex=True,
).str.strip()

# Remove transferred holidays
df_holi = df_holi[df_holi['transferred'] == False]

# Extract work days
work_days = df_holi[df_holi.type.eq("Work Day")]
work_days = work_days[["date", "type"]].rename(
    columns={"type": "work_day"}
).reset_index(drop=True)
work_days.work_day = work_days.work_day.notna().astype(int)

hol_df = df_holi[df_holi['type'] != "Work Day"].reset_index(drop=True)

# Extract local holidays
local_holidays = df_holi[df_holi['locale'] == 'Local']
local_holidays = local_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "city"}
).reset_index(drop=True)

local_holidays = pd.get_dummies(local_holidays, columns=["description"], prefix="loc")

# Extract regional/state holidays
regional_holidays = df_holi[df_holi['locale'] == 'Regional']
regional_holidays = regional_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "state", "description": "provincializacion"}
).reset_index(drop=True)
regional_holidays.provincializacion = regional_holidays.provincializacion.eq("provincializacion").astype(int)

# Extract national holidays
national_holidays = df_holi[df_holi['locale'] == 'National']
national_holidays = national_holidays[["date", "description"]].reset_index(drop=True)
national_holidays = national_holidays[~national_holidays.duplicated()]
national_holidays = pd.get_dummies(national_holidays, columns=["description"], prefix="nat")
# Different national holidays may fall on the same day
national_holidays = national_holidays.groupby("date").sum().reset_index()
# Shorten name for visualization purposes later
national_holidays = national_holidays.rename(columns={"nat_primer grito independencia": "nat_primer grito"})

In [ ]:
# Check for missing dates in test dataset
test_start = df_test.date.min()
test_end = df_test.date.max()

print('Dates missing from test dataset: ', pd.date_range(test_start, test_end).difference(df_test.date.unique()))

# 4. Compile datasets

In [ ]:
# Compile dfs
def compile_dfs(df):
    temp_df = df.merge(
        df_trans, on=["date", "store_nbr"], how="left",
    ).merge(
        df_oil, on="date", how="left",
    ).merge(
        df_stores, on="store_nbr", how="left",
    ).merge(
        work_days, on="date", how="left",
    ).merge(
        local_holidays, on=["date", 'city'], how="left",
    ).merge(
        regional_holidays, on=["date", 'state'], how="left",
    ).merge(
        national_holidays, on="date", how="left",
    ).sort_values(["date", "store_nbr", "family"], ignore_index=True)

    # Fill columns with 0s to indicate absence of holidays/events
    return(temp_df.fillna(0))


df_trainNew = compile_dfs(df_train)
df_testNew = compile_dfs(df_test)

# 5. Additional analysis

In [ ]:
# Are holidays significant to sales?
# Most are!
def AB_Test(dataframe, group, target):
    
    # Packages
    from scipy.stats import shapiro
    import scipy.stats as stats
    
    # Split A/B
    groupA = dataframe[dataframe[group] == 1][target]
    groupB = dataframe[dataframe[group] == 0][target]
    
    # Assumption: Normality
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05
    # H0: Distribution is Normal! - False
    # H1: Distribution is not Normal! - True

    if (ntA == False) & (ntB == False): # "H0: Normal Distribution"
        # Parametric Test
        # Assumption: Homogeneity of variances
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        # H0: Homogeneity: False
        # H1: Heterogeneous: True
        
        if leveneTest == False:
            # Homogeneity
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            ttest = stats.ttest_ind(groupA, groupB, equal_var=False)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(groupA, groupB)[1] 
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True
        
    # Result
    temp = pd.DataFrame({
        "AB Hypothesis":[ttest < 0.05], 
        "p-value":[ttest]
    })
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!", "A/B groups are not similar!")
    temp["Feature"] = group
    temp["GroupA_mean"] = groupA.mean()
    temp["GroupB_mean"] = groupB.mean()
    temp["GroupA_median"] = groupA.median()
    temp["GroupB_median"] = groupB.median()
    
    # Columns
    if (ntA == False) & (ntB == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Feature","Test Type", "Homogeneity","AB Hypothesis", "p-value", "Comment", "GroupA_mean", "GroupB_mean", "GroupA_median", "GroupB_median"]]
    else:
        temp = temp[["Feature","Test Type","AB Hypothesis", "p-value", "Comment", "GroupA_mean", "GroupB_mean", "GroupA_median", "GroupB_median"]]
    
    # Print Hypothesis
    # print("# A/B Testing Hypothesis")
    # print("H0: A == B")
    # print("H1: A != B", "\n")
    
    return temp
    
# Apply A/B Testing
he_cols = df_trainNew.columns[12:].tolist()
ab = []
for i in he_cols:
    ab.append(AB_Test(dataframe=df_trainNew[df_trainNew.sales.notnull()], group = i, target = "sales"))
ab = pd.concat(ab)
ab

In [ ]:
# QQ Plots - What is data distribution?
import statsmodels.graphics.gofplots as sm 

for i in he_cols:
    print(i)
    df_trainNewPlotTmp = df_trainNew[df_trainNew['sales'].notnull()]
    df_trainNewPlotTmp1 = df_trainNewPlotTmp[df_trainNewPlotTmp[i] == 1]['sales']
    df_trainNewPlotTmp2 = df_trainNewPlotTmp[df_trainNewPlotTmp[i] == 0]['sales']
    print("length of df1:", len(df_trainNewPlotTmp1))
    print("length of df2:", len(df_trainNewPlotTmp2))

    fig1, ax = plt.subplots(1, 2, figsize=(12, 7)) 
    sns.histplot(df_trainNewPlotTmp1, kde=True, color ='blue',ax=ax[0]) 
    sm.ProbPlot(df_trainNewPlotTmp1).qqplot(line='s', ax=ax[1])

    fig2, ax = plt.subplots(1, 2, figsize=(12, 7)) 
    sns.histplot(df_trainNewPlotTmp2, kde=True, color ='blue',ax=ax[0]) 
    sm.ProbPlot(df_trainNewPlotTmp1).qqplot(line='s', ax=ax[1])

    plt.show(block=False)
    plt.pause(3)
    plt.close()